In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
tf.__version__

In [ ]:
train_dir = 'dataset_balanceado_final/train'
validation_dir = 'dataset_balanceado_final/validation'
test_dir = 'dataset_balanceado_final/test'

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

IMG_SIZE = 150
BATCH_SIZE = 32

In [ ]:
train_dataset = image_dataset_from_directory(
    train_dir, 
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical' 
)

In [ ]:
validation_dataset = image_dataset_from_directory(
    validation_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

In [ ]:
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

In [ ]:
# Utiliza uma função(do sckicit-learn) para avaliar o desempenho do modelo, indicando Métricas como: 
    # f1-score do modelo
    # accuracy do modelo
    # accuracy por classe 

from sklearn.metrics import classification_report
import numpy as np

def print_classification_metrics(model, dataset, phase_name):
    y_true = []
    y_pred = []

    for images, labels in dataset:
        preds = model.predict(images)
        y_true.extend(np.argmax(labels.numpy(), axis=1))
        y_pred.extend(np.argmax(preds, axis=1))

    print(f"\n {phase_name}")
    print(classification_report(y_true, y_pred, digits=4))


In [ ]:
from tensorflow import keras 
from keras import layers 
from keras.applications import VGG16 # Importa a arquitetura VGG16 pré-treinada do Keras.

# Data augmentation
data_augmentation_vgg16 = keras.Sequential([ 
    layers.RandomFlip("vertical"),# Aplica inversão vertical nas imagens.
    layers.RandomFlip("horizontal"), # Aplica inversão horizontal nas imagens
    layers.RandomTranslation(0.1, 0.2), 
    layers.RandomRotation(0.4), # Aplica rotação de 20% 
    #layers.RandomZoom(0.1)
    ]) # Aplica zoom de 20% 

# Carregar a base VGG16 pré-treinada
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3)) # Carrega o modelo VGG16 pré-treinado no ImageNet, sem a camada do topo
conv_base.trainable = False # Congela todas as camadas da VGG16, impedindo que os seus pesos sejam atualizados durante o treino (feature extraction).

# Usar data augmentation
inputs = layers.Input(shape=(150, 150, 3)) # Define a camada de entrada do novo modelo com o formato das imagens e 3 canais (RGB)
x = data_augmentation_vgg16(inputs) # Aplica as transformações de aumento de dados nas imagens de entrada.
x = keras.applications.vgg16.preprocess_input(x) # Aplica o pré-processamento específico da VGG16 
x = conv_base(x) # Passa as imagens (pré-processadas e aumentadas) através da base VGG16 congelada para extrair características.
x = layers.Flatten()(x) # Achata as características extraídas para um vetor 1D.
x = layers.Dense(256, activation='relu')(x) # Adiciona uma camada densa com 256 neurónios e ativação ReLU.
x = layers.Dropout(0.5)(x) # Aplica Dropout (50%) para regularização e prevenção de overfitting.
outputs = layers.Dense(7, activation='softmax')(x) # Adiciona a camada de saída densa com 7 neurónios (para 7 classes) e ativação Softmax

model_t = models.Model(inputs, outputs) # Cria o model_t

# Compilar e treinar (feature extraction)
model_t.compile( # Compila o modelo para configurar o processo de treino.
    loss='categorical_crossentropy', # Define a função de loss categorical crossentropy.
    optimizer=keras.optimizers.Adam(learning_rate=1e-3), # Configura o otimizador Adam  learning_rate=1e-3) 
    metrics=['accuracy'] # Define 'accuracy' (precisão) como a métrica a ser monitorizada.
)

history_t = model_t.fit( # Treina o modelo.
    train_dataset, # Usa o conjunto de dados de treino.
    validation_data=validation_dataset, # Usa o conjunto de dados de validação para monitorizar o desempenho.
    epochs=15 # Treina o modelo por 10 épocas.
)

In [ ]:
print_classification_metrics(model_t, test_dataset, "Modelo 2 : VGG16 (Feature Extraction com Augmentation)")

In [ ]:
import matplotlib.pyplot as plt
accuracy = history_t.history['accuracy']
val_acc = history_t.history['val_accuracy']
loss = history_t.history['loss']
val_loss = history_t.history['val_loss']
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# Descongelar parte da VGG16 (últimas camadas)
conv_base.trainable = True
for layer in conv_base.layers[:-4]:  # Descongelar as ultimas -8
    layer.trainable = False # manter as primeiras camadas congeladas

# Recompilar o modelo com learning rate menor
model_t.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

# Treinar novamente
history_t = model_t.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=15 
)

In [ ]:
print_classification_metrics(model_t, test_dataset, "Modelo 2 : Fine-tuning")

In [ ]:
import matplotlib.pyplot as plt
accuracy = history_t.history['accuracy']
val_acc = history_t.history['val_accuracy']
loss = history_t.history['loss']
val_loss = history_t.history['val_loss']
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Obter predições no test_dataset
y_true = []
y_pred = []

for images, labels in test_dataset:
    preds = model_t.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

class_names = test_dataset.class_names  

# Criar e mostrar a matriz de confusão
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45, values_format='d')
plt.title("Matriz de Confusão - Teste")
plt.tight_layout()
plt.show()

In [ ]:
model_t.save("modelT_3A_com_data_aug_adam_cat_cross_best_acc.keras")

In [ ]:
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input

def extract_computed_features(conv_base, dataset, augment_layer=None):
    all_features = []
    all_labels = []

    for images, labels in dataset:
        # Aplica data augmentation se fornecido
        if augment_layer:
            images = augment_layer(images)
        
        # Pre-processamento obrigatório para VGG16
        preprocessed = preprocess_input(images)
        
        # Extrair as computed features
        features = conv_base.predict(preprocessed)
        
        all_features.append(features)
        all_labels.append(labels)

    # Junta tudo num único array
    features_array = np.concatenate(all_features)
    labels_array = np.concatenate(all_labels)

    return features_array, labels_array


In [ ]:
train_features, train_labels = extract_computed_features(conv_base, train_dataset, augment_layer=data_augmentation_vgg16)
val_features, val_labels = extract_computed_features(conv_base, validation_dataset, augment_layer=None)
test_features, test_labels = extract_computed_features(conv_base, test_dataset, augment_layer=None)


In [ ]:
from numpy import save

save("modelt_3A_train_features.npy", train_features)
save("modelt_3A_train_labels.npy", train_labels)

save("modelt_3A_val_features.npy", val_features)
save("modelt_3A_val_labels.npy", val_labels)

save("modelt_3A_test_features.npy", test_features)
save("modelt_3A_test_labels.npy", test_labels)
